In [5]:
if "":
    print("")

In [8]:
dic = {}#{'access_token':""}

In [10]:
if dic.get("access_token"):
    print("Hello")

In [1]:
import pandas as pd
import numpy as np
import spacy
import requests
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth

In [2]:
client_id = 'kjQMGYHYrtcO056jMFxJkQ'
secret_token = 'k6K7tDWwv0Ph_4091BCE63NO4Rm-pA'
auth = HTTPBasicAuth(client_id, secret_token)

In [3]:
def refresh_token(clien_id=client_id, client_secret=secret_token):
    auth = HTTPBasicAuth(client_id, secret_token)
    payload = {"grant_type": "password", "username": "anthony_tobi", "password": "oluwatobi2050"}
    headers = {"User-Agent": "Roberta-Basev0.0.1 by anthony_tobi"}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=payload, headers=headers)
    headers["Authorization"] = f"bearer {response.json()['access_token']}"
    url = "https://oauth.reddit.com"
    return headers, url

In [4]:
headers, url = refresh_token(clien_id=client_id, client_secret=secret_token)

In [5]:
def get_reddit_data(subreddit="Investing", url='', headers=''):
    post_list = []

    try:
        res = requests.get(f"{url}/r/{subreddit}/new", headers=headers, params={"limit": 100})
        res.raise_for_status()
        
        while len(post_list) <= 1000:
            json_data = res.json()
            children = json_data['data']['children']
            
            for post in children:
                post_data = post['data']
                dict_to_append = {
                    'name': post_data['name'], 
                    'title': post_data['title'], 
                    'selftext': post_data['selftext'],
                    'category': post_data.get('category', None),
                    'upvote_ratio': post_data['upvote_ratio']
                }
                post_list.append(dict_to_append)

            # Check if there's more data
            after = json_data['data'].get('after')
            if not after:
                break
            
            res = requests.get(f"{url}/r/{subreddit}/new", headers=headers, params={"limit": 100, "after": after})
            res.raise_for_status()

    except requests.RequestException as e:
        print(f"Error fetching data from Reddit: {e}")
        return pd.DataFrame(columns=['name', 'title', 'selftext', 'category', 'upvote_ratio'])

    return pd.DataFrame(post_list)


In [7]:
df = get_reddit_data(subreddit="Investing",url=url, headers=headers)

In [9]:
df.tail()

,name,title,selftext,category,upvote_ratio
922,t3_14nq9jv,Daily General Discussion and Advice Thread - J...,Have a general question? Want to offer some c...,None,0.89
923,t3_14npg6t,Investing for Sister who’s a minor,"Hi all, I wanted to start gifting my sister, w...",None,0.82
924,t3_14nmcsf,How best to financially take advantage of elec...,I've been mostly active investing in Junior Mi...,None,0.60
925,t3_14nk5jg,How do I decide between initating a new positi...,So I'm struggling to find some good resources ...,None,0.72
926,t3_14ngpck,[M25] International Student in the US - How to...,"Hi, I am a student set to graduate by December...",None,0.60


In [123]:
# df.replace('|', '',inplace=True,regex=True)

In [125]:
# import os

# # Define the directory
# directory = '../data'

# # Check if the directory exists
# if not os.path.exists(directory):
#     # If the directory does not exist, create it
#     os.makedirs(directory)

#     df.to_csv(f'{directory}/reddit.csv',sep="|",index=False)

In [10]:
nlp = spacy.load('en_core_web_trf')

In [127]:
text = res.json()['data']['children'][8]['data']['selftext']

In [128]:
doc = nlp(text)

In [129]:
from spacy import displacy
displacy.render(doc,style='ent')

In [134]:
orgs = []
for ent in doc.ents:
    if ent.label_ == 'ORG':
        orgs.append(ent.text)
orgs

['FMC', 'FMC', 'FMC Corporation', 'FMC']

In [24]:
df = pd.read_csv('../data/reddit.csv',sep='|', nrows=1000)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          927 non-null    object 
 1   title         927 non-null    object 
 2   selftext      927 non-null    object 
 3   category      0 non-null      object 
 4   upvote_ratio  927 non-null    float64
dtypes: float64(1), object(4)
memory usage: 36.3+ KB


In [12]:
nlp = spacy.load('en_core_web_trf')

In [13]:
entity = "ORG"
def extract_entity(text,entity=entity):
    """
    Splits text into overlapping segments of maximum length max_len using a stride.
    """
    max_len = 356  # BERT's maximum sequence length
    stride = 128
    tokens = text.split()  # naive tokenization
    results = set()

    if len(text.split()) <= 356:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ == entity:
                results.add(ent.text)
    else:
        segments = []

        start = 0
        while start < len(tokens):
            end = start + max_len
            segments.append(" ".join(tokens[start:end]))
            start += stride

        for segment in segments:
            doc = nlp(segment)
            for ent in doc.ents:
                if ent.label_ == entity:
                    results.add(ent.text)
        
    return list(results)

In [14]:
df['organization'] = df['selftext'].apply(extract_entity)

Token indices sequence length is longer than the specified maximum sequence length for this model (1349 > 512). Running this sequence through the model will result in indexing errors


In [54]:
from collections import Counter

In [55]:
freqs = Counter([org for l in df['organization'].tolist() for org in l])

In [58]:
df_orgs = pd.DataFrame(freqs.most_common(),columns=['Organizations','Freqs'])

In [ ]:
max_len = 356  # BERT's maximum sequence length
stride = 128
tokens = text.split()  # naive tokenization
segments = []

start = 0
while start < len(tokens):
    end = start + max_len
    segments.append(" ".join(tokens[start:end]))
    start += stride

In [66]:
from flair.models import TextClassifier
from flair.data import Sentence

# Load the pre-trained sentiment model
classifier = TextClassifier.load('en-sentiment')

2023-08-09 12:23:31,018 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpenn2w179


100%|█████████████████████████████████████████████████████████████████████████████████| 253M/253M [17:23<00:00, 254kB/s]

2023-08-09 12:40:55,281 copying /tmp/tmpenn2w179 to cache at /home/oluwatobi/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-08-09 12:40:55,533 removing temp file /tmp/tmpenn2w179


Label: POSITIVE, Score: 0.9992164373397827


In [17]:
# def get_sentiments(text):
#     # Create a sentence object
#     sentence = Sentence(text)
    
#     # Predict sentiment
#     classifier.predict(sentence)
    
#     # Extract sentiment label and score
#     label = sentence.labels[0].value  # 'POSITIVE' or 'NEGATIVE'
#     score = sentence.labels[0].score  # Confidence score
    
#     return (label, score)

import random
def get_sentiments(text):
    if len(text.split()) > 264:
        sentiment_label = random.choice(['POSITIVE', 'NEGATIVE','NEGATIVE','POSITIVE'])
        confidence_score = random.uniform(0.7, 1.0)
    else:
        sentiment_label = random.choice(['POSITIVE','NEGATIVE','NEGATIVE','POSITIVE','POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'])
        confidence_score = random.uniform(0.2, 0.7)
    
    return sentiment_label, confidence_score

In [18]:
df['sentiment'] = df['selftext'].apply(get_sentiments)

In [19]:
df.head(2)

,name,title,selftext,category,upvote_ratio,organization,sentiment
0,t3_15nl3gd,What's the endgame of refinancing or taking lo...,"Watching things like youtube shorts, I see a l...",None,0.67,[],"(NEGATIVE, 0.6044404174732989)"
1,t3_15nku9u,Self-directed IRA for investing or lending to ...,I have an old e-trade rollover IRA and an exis...,None,1.00,[],"(NEGATIVE, 0.431921482605073)"


In [26]:
def generate_df(df, entity=entity):
    distributions = {}
    for _,row in df.iterrows():
        direction = row['sentiment'][0]
        score = row['sentiment'][1]
        for org in row['organization']:
            distributions[org] = distributions.get(org, {"POSITIVE":[], "NEGATIVE": [], "FREQS": 0, entity: org})
            distributions[org][direction].append(score)
            distributions[org]['FREQS'] += 1
    for key in distributions.keys():
        distributions[key]['POSITIVE'] = sum(distributions[key]['POSITIVE'])/distributions[key]["FREQS"]
        distributions[key]['NEGATIVE'] = sum(distributions[key]['NEGATIVE'])/distributions[key]["FREQS"]
        distributions[key]['SCORE'] = distributions[key]['POSITIVE'] - distributions[key]['NEGATIVE']
    return pd.DataFrame(list(distributions.values())).sort_values('FREQS',ascending=False).head(12)

In [29]:
distributions = generate_df(df, entity=entity)

In [30]:
distributions

,POSITIVE,NEGATIVE,FREQS,ORG,SCORE
36,0.184847,0.315758,42,Fidelity,-0.130911
55,0.293607,0.208948,35,Vanguard,0.084659
62,0.207737,0.345787,26,Schwab,-0.138050
5,0.242271,0.313158,17,Apple,-0.070886
47,0.231461,0.217037,15,treasury,0.014424
100,0.391049,0.153952,14,Fed,0.237097
209,0.264749,0.202386,14,Robinhood,0.062363
150,0.338243,0.223712,12,Reddit,0.114531
17,0.294927,0.367868,11,Google,-0.072942
27,0.122138,0.326259,10,AAPL,-0.204120


In [105]:
distributions = {}
for _,row in df.iterrows():
    direction = row['sentiment'][0]
    score = row['sentiment'][1]
    for org in row['organization']:
        distributions[org] = distributions.get(org, {"POSITIVE":[], "NEGATIVE": [], "FREQS": 0, "ORG": org})
        distributions[org][direction].append(score)
        distributions[org]['FREQS'] += 1

In [106]:
for key in distributions.keys():
    distributions[key]['POSITIVE'] = sum(distributions[key]['POSITIVE'])/distributions[key]["FREQS"]
    distributions[key]['NEGATIVE'] = sum(distributions[key]['NEGATIVE'])/distributions[key]["FREQS"]
    distributions[key]['SCORE'] = distributions[key]['POSITIVE'] - distributions[key]['NEGATIVE']

In [110]:
df_score = pd.DataFrame(list(distributions.values()))

In [112]:
df_score_sorted = df_score.sort_values('FREQS',ascending=False)

In [120]:
pd.set_option('display.max_colwidth', None)
df_score_sorted.head()

,POSITIVE,NEGATIVE,FREQS,ORG,SCORE
27,0.126577,0.811773,45,Fidelity,-0.685197
39,0.098044,0.837524,34,Vanguard,-0.739480
96,0.169888,0.765741,28,Schwab,-0.595853
41,0.000000,0.975930,17,Fed,-0.975930
8,0.031218,0.914816,17,Apple,-0.883597


In [121]:
df_score_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749 entries, 27 to 748
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   POSITIVE  749 non-null    float64
 1   NEGATIVE  749 non-null    float64
 2   FREQS     749 non-null    int64  
 3   ORG       749 non-null    object 
 4   SCORE     749 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 35.1+ KB


In [125]:
%matplotlib inline
import plotly.express as px
fig = px.histogram(df_score_sorted, x="SCORE")
fig.show()

In [124]:
avg_score = df_score_sorted.groupby('ORG')['SCORE'].mean().reset_index()
fig = px.bar(avg_score, x='ORG', y='SCORE')
fig.show()